In [1]:
%load_ext autoreload
%autoreload 2
%xmode Plain

Exception reporting mode: Plain


In [31]:
import pandas as pd
import plotly
import plotly.express as px 
import dtale
import numpy as np
import random
from bazaraki import utils
from tqdm import tqdm
import swifter
from pathlib import Path
import datacompy
from datetime import datetime, date
from parse import parse
from glob import glob 


In [3]:
tqdm.pandas()
pd.set_option('display.max_rows', 100)  # Disable row limit
pd.set_option('display.max_columns', 60)  # Disable column limit
pd.set_option('display.width', 20)  # Disable line width limit
pd.set_option('display.max_colwidth', 100)  # Disable column width limit
pd.set_option('display.precision', 2)  
pd.set_option('display.float_format', '{:.4f}'.format)  
np.random.seed(42)
random.seed(42)
pd.options.plotting.backend = "plotly"
plotly.io.renderers.default = "notebook_connected"
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [91]:
DT_LEN = len("2024-12-12 18:34:25")
merged = pd.DataFrame()
for file_name in sorted(glob("output/*.jsonl")):
    print(f"Reading {file_name}")
    result = parse("output/{date} {time} {postfix}", file_name)
    if result is None:
        print(f"Failed to parse {file_name}")
        continue
    if result["postfix"].startswith("fast"):
        print(f"Skipping fast run")
        continue
    dt = date.fromisoformat(result["date"])
    
    newdf = pd.read_json(file_name, lines=True)
    assert newdf.ad_id.duplicated().sum() == 0, "Expected no duplicates"
    newdf.set_index("ad_id", inplace=True)
    if merged.empty:
        merged = newdf
        continue
    
    deleted = merged.index.difference(newdf.index)
    merged.loc[deleted, "delete_date"] = dt
    
    new = newdf.index.difference(merged.index)
    merged = pd.concat([merged, newdf.loc[new]])
    print(f"Total: {len(merged)} read: {len(newdf)} new: {len(new)} deleted: {len(deleted)}")
    

Reading output/2024-12-12 18:34:25 real-estate-to-rent_real-estate-for-sale.jsonl


Reading output/2024-12-14 11:44:22 real-estate-to-rent_real-estate-for-sale.jsonl
Total: 35559 read: 34669 new: 745 deleted: 890
Reading output/2024-12-15 18:00:14 real-estate-to-rent_real-estate-for-sale.jsonl
Total: 35734 read: 34398 new: 175 deleted: 1336
Reading output/2024-12-15 20:17:12 fast_real-estate-to-rent_real-estate-for-sale.jsonl
Skipping output/2024-12-15 20:17:12 fast_real-estate-to-rent_real-estate-for-sale.jsonl


In [74]:
merged["price_sqm"] = merged.price / (merged["Property area"]).round(2)

In [75]:
def add_city_disctrict_cols(df):
    df[["city", "district"]] = df.location.str.split(",", expand=True)
    return df
merged = add_city_disctrict_cols(merged)


In [76]:
deleted = merged.query("not delete_date.isna() and url.str.contains('apartment-to-rent')")
deleted

,url,title,price,original_price,price_per_sqm,location,posted,reference_number,views,lat,lng,sold,cat0,cat1,Property area,Pets,Type,Parking,Plot area,Furnishing,Included,Online viewing,Air conditioning,Energy Efficiency,Bedrooms,Bathrooms,images,description,Floor,Postal code,Construction year,Reference number,Condition,Square meter price,Minimum stay,Land type,Plot Type,Parcel number,Planning zone,Registration number,Share,Density,Coverage,Registration block,Area,Pick a point,delete_date,price_sqm,city,district
ad_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5564097,https://www.bazaraki.com/adv/5564097_1-bedroom-apartment-to-rent/,1-bedroom apartment to rent,1250,NaN,NaN,"Limassol, Polemidia Kato",09.12.2024 07:41,NaN,23,34.6941,33.0139,False,Cyprus real estate to rent,"Apartments, flats to rent",60.0000,Not allowed,Apartment,Covered,NaN,Fully Furnished,"Elevator, Storage room",No,"Full, all rooms",B+,1,1,"[https://cdn1.bazaraki.com/media/cache1/57/33/5733f01732bea3000e0e7321b5b7ba42.webp, https://cdn...",1 bedroom\n1 bathroom\n60m2 total area\nBalcony\nCovered parking\n1st floor\nFully furnished\nEl...,1st,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-12-15,20.8333,Limassol,Polemidia Kato
5516998,https://www.bazaraki.com/adv/5516998_3-bedroom-apartment-to-rent/,3-bedroom apartment to rent,900,NaN,NaN,"Nicosia, Strovolos - Stavros",09.12.2024 11:48,NaN,122,35.1313,33.3524,False,Cyprus real estate to rent,"Apartments, flats to rent",210.0000,Not allowed,Apartment,Uncovered,NaN,Semi-Furnished,Balcony,NaN,"Full, all rooms",B,3,3,"[https://cdn1.bazaraki.com/media/cache1/60/68/6068ef53b460177affae187eda047c34.webp, https://cdn...","FOR RENT!!! Well-maintained 3-bedroom ,210sq.m flat in Strovolos!!!\nMonthly rent €900(including...",1st,NaN,1996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-12-15,4.2857,Nicosia,Strovolos - Stavros
5564437,https://www.bazaraki.com/adv/5564437_2-bedroom-apartment-to-rent/,2-bedroom apartment to rent,950,NaN,NaN,"Paphos, Paphos - Universal",09.12.2024 12:30,NaN,31,34.7630,32.4218,False,Cyprus real estate to rent,"Apartments, flats to rent",95.0000,Not allowed,Apartment,Uncovered,NaN,Fully Furnished,"Pool, Balcony",No,"Full, all rooms",A,2,2,"[https://cdn1.bazaraki.com/media/cache1/05/bb/05bb1d5893aa4fc4ab70160e437495d3.webp, https://cdn...",2 bedroom maisonette in Universal Areal with all electric device and air-condition in all room.....,Ground floor,NaN,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-12-15,10.0000,Paphos,Paphos - Universal
5502841,https://www.bazaraki.com/adv/5502841_2-bedroom-apartment-to-rent/,2-bedroom apartment to rent,1350,1600.0000,NaN,"Limassol, Germasogeia Tourist Area",09.12.2024 13:38,NaN,53,34.6933,33.0820,False,Cyprus real estate to rent,"Apartments, flats to rent",80.0000,Not allowed,Apartment,Covered,NaN,Fully Furnished,"Garden, Elevator, Balcony",No,"Full, all rooms",N/A,2,1,"[https://cdn1.bazaraki.com/media/cache1/cd/20/cd2000ffd39d198f5f9d5e2bb9410bc0.webp, https://cdn...",A newly refurbished 2 bedroom apartment situated on the second floor of a small 4 floor building...,2nd,4047.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-12-15,16.8750,Limassol,Germasogeia Tourist Area
5433835,https://www.bazaraki.com/adv/5433835_2-bedroom-apartment-to-rent/,2-bedroom apartment to rent,5500,NaN,NaN,"Limassol, Agios Tychon Tourist Area",09.12.2024 10:49,NaN,181,34.7106,33.1337,False,Cyprus real estate to rent,"Apartments, flats to rent",114.0000,NaN,Apartment,NaN,NaN,Fully Furnished,"Pool, Garden, Elevator, Balcony",NaN,"Full, all rooms",N/A,2,NaN,"[https://cdn1.bazaraki.com/media/cache1/b1/95/b195dbb8761897e9648d18b4a6d58c8d.webp, https://cdn...",For rent 2 bdr apartment\n2 months payment in advance.\nExperience luxurious waterfront living i...,7th,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-12-15,48.2456,Limassol,Agios Tychon Tourist Area
...,...,...

In [90]:
# rent speed total
n_total = merged.groupby("city").size()
n_deleted = merged.query("not delete_date.isna() and url.str.contains('apartment-to-rent')").groupby("city").size()
speed = n_deleted / n_total 
pd.concat([speed, n_total, n_deleted], axis=1, keys=["speed", "n_total", "n_deleted"]).sort_values("speed", ascending=False)

,speed,n_total,n_deleted
city,,,
Nicosia,0.0121,7004,85
Limassol,0.0095,16659,159
Paphos,0.0074,4183,31
Larnaca,0.0060,6295,38
Famagusta,0.0019,1593,3


In [70]:
merged.price.hist().show()

In [7]:
df1 = pd.read_json("output/2024-12-12 18:34:25 real-estate-to-rent_real-estate-for-sale.jsonl", lines=True)

In [8]:
df2 = pd.read_json("output/2024-12-14 11:44:22 real-estate-to-rent_real-estate-for-sale.jsonl", lines=True)

In [9]:
df1.set_index("ad_id", inplace=True)
df2.set_index("ad_id", inplace=True)

In [11]:
largedf

,url,title,price,original_price,price_per_sqm,location,posted,reference_number,views,lat,lng,sold,cat0,cat1,Property area,Pets,Type,Parking,Plot area,Furnishing,Included,Online viewing,Air conditioning,Energy Efficiency,Bedrooms,Bathrooms,images,description,Floor,Postal code,Construction year,Reference number,Condition,Square meter price,Minimum stay,Land type,Plot Type,Parcel number,Planning zone,Registration number,Share,Density,Coverage,Registration block,Area,Pick a point
ad_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5415213,https://www.bazaraki.com/adv/5415213_4-bedroom-detached-house-to-rent/,4-bedroom detached house to rent,1650,NaN,NaN,"Larnaca, Dromolaxia",Yesterday 20:57,NaN,472,34.8703,33.5940,False,Cyprus real estate to rent,Houses to rent,218.0000,Not allowed,Detached house,Uncovered,542.0000,Fully Furnished,"Garden, Balcony",Yes,"Full, all rooms",N/A,4,2,"[https://cdn1.bazaraki.com/media/cache1/af/7a/af7a55295335956b16be66e00547ef40.webp, https://cdn...",This spacious four-bedroom house is located in a quiet neighborhood and is perfect for families ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5508233,https://www.bazaraki.com/adv/5508233_2-bedroom-apartment-to-rent/,2-bedroom apartment to rent,1150,NaN,NaN,"Paphos, Chlorakas",Yesterday 21:02,NaN,385,34.7904,32.4153,False,Cyprus real estate to rent,"Apartments, flats to rent",100.0000,Allowed,Apartment,Uncovered,NaN,Fully Furnished,"Pool, Balcony",No,"Full, all rooms",A,2,1,"[https://cdn1.bazaraki.com/media/cache1/55/3c/553c55844092d64dcf6b81b88fdeaa83.webp, https://cdn...",This beautiful 2 bedroom apartment is located on a hillside in the area of Paphos which has uns...,2nd,8220.0000,2008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5395481,https://www.bazaraki.com/adv/5395481_2-bedroom-apartment-to-rent/,2-bedroom apartment to rent,1100,NaN,NaN,"Larnaca, Livadia Larnakas",07.12.2024 11:07,NaN,26,34.9548,33.6378,False,Cyprus real estate to rent,"Apartments, flats to rent",90.0000,Not allowed,Apartment,Covered,NaN,Fully Furnished,"Pool, Elevator, Balcony",Yes,"Full, all rooms",A,2,2,"[https://cdn1.bazaraki.com/media/cache1/26/c8/26c8d90c413acdce05615bf5b068e203.webp, https://cdn...","This is a 2 bedroom, 2 bathroom flat in a beautifully refurbished complex. The flat has been new...",1st,7060.0000,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5562261,https://www.bazaraki.com/adv/5562261_1-bedroom-apartment-to-rent/,1-bedroom apartment to rent,720,NaN,NaN,"Nicosia, Nicosia - Ag. Antonios",07.12.2024 12:34,NaN,45,35.1725,33.3650,False,Cyprus real estate to rent,"Apartments, flats to rent",60.0000,Not allowed,Apartment,Covered,NaN,Unfurnished,NaN,NaN,"Full, all rooms",B,1,1,"[https://cdn1.bazaraki.com/media/cache1/93/04/93042a9a5112ec854815cc86678202cd.webp, https://cdn...",Rent this beautiful Flat through www.rentonline.today and save 30% on your first monthly rent!. ...,1st,NaN,2015,rentonline-107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5311477,https://www.bazaraki.com/adv/5311477_6-bedroom-detached-house-for-sale/,6-bedroom detached house fоr sаle,5900000,NaN,7338.0000,"Limassol, Agios Athanasios",29.11.2024 19:00,NaN,2233,34.7339,33.0545,False,Cyprus real estate for sale,Houses for sale,804.0000,NaN,Detached house,Covered,15456.0000,NaN,"Garden, Alarm, Fireplace, Balcony, Playroom, Attic/Loft, Storage room",NaN,"Full, all rooms",C,6,5+,"[https://cdn1.bazaraki.com/media/cache1/0c/d0/0cd0c1184905eccd18de9101fa3df71d.webp, https://cdn...","This stunning property is truly one-of-a-kind, offering a spacious 6-bedroom villa spanning an i...",NaN,4108.0000,2000,NaN,Resale,€7.338 /,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5571776,https://www.bazaraki.com/adv/5571776_2-bedroom-apartment-for-sale/,

In [ ]:
new_indices = df2.index.difference(df1.index)
print(f"{len(new_indices)} new records")
largedf = pd.concat([df1, df2.loc[new_indices]])
len(largedf)

745 new records


35559

In [46]:
for i, r1 in df1.iterrows():
    try:
        r2 = df2.loc[i]
    except KeyError:
        #print(f"KeyError: {i}")
        continue
    if r1.title != r2.title:
        print(f"df1 {i} {r1.url} {r1.title} {r1.posted}")
        print(f"df2 {i} {r2.url} {r2.title} {r2.posted}")


df1 5527956 https://www.bazaraki.com/adv/5527956_4-bedroom-detached-house-for-sale/ 4-bedroom detached house fоr sаle 13.11.2024 18:12
df2 5527956 https://www.bazaraki.com/adv/5527956_4-bedroom-detached-house-for-sale/ 3-bedroom detached house fоr sаle 12.12.2024 20:52
df1 5185306 https://www.bazaraki.com/adv/5185306_residential-land-5342-m2/ Residential land 5342 m² 25.11.2024 02:38
df2 5185306 https://www.bazaraki.com/adv/5185306_residential-land-5342-m2/ Residential land 627 m² 25.11.2024 02:38
df1 5100938 https://www.bazaraki.com/adv/5100938_1-bedroom-apartment-for-sale/ 1-bedroom apartment fоr sаle 23.11.2024 17:45
df2 5100938 https://www.bazaraki.com/adv/5100938_1-bedroom-apartment-for-sale/ 2-bedroom apartment fоr sаle 23.11.2024 17:45
df1 5548384 https://www.bazaraki.com/adv/5548384_2-bedroom-apartment-for-sale/ 2- Bedroom apartment in Laiki Lefkothea with 2 storage rooms and 2 ... 27.11.2024 14:23
df2 5548384 https://www.bazaraki.com/adv/5548384_2-bedroom-apartment-for-sale/ 2

In [34]:
df1.loc[5564097]

url                                                      https://www.bazaraki.com/adv/5564097_1-bedroom-apartment-to-rent/
title                                                                                          1-bedroom apartment to rent
price                                                                                                                 1250
original_price                                                                                                         NaN
price_per_sqm                                                                                                          NaN
location                                                                                          Limassol, Polemidia Kato
posted                                                                                                    09.12.2024 07:41
reference_number                                                                                                       NaN
views           

In [35]:
df2.loc[5564097]

KeyError: 5564097

In [36]:
import pandas as pd

# Example DataFrames
df1 = pd.DataFrame({'A': [1, 2]}, index=[1, 2])
df2 = pd.DataFrame({'B': [3, 4]}, index=[2, 3])

# Union of indexes
union_indexes = df1.index.union(df2.index)

print("Union of indexes:", union_indexes)

Union of indexes: Index([1, 2, 3], dtype='int64')


In [18]:
print(datacompy.Compare(df1, df2, on_index=True, df1_name='df1', df2_name='df2').report())

DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns   Rows
0       df1       46  34814
1       df2       46  34669

Column Summary
--------------

Number of columns in common: 46
Number of columns in df1 but not in df2: 0 []
Number of columns in df2 but not in df1: 0 []

Row Summary
-----------

Matched on: index
Any duplicates on match values: No
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 33,924
Number of rows in df1 but not in df2: 890
Number of rows in df2 but not in df1: 745

Number of rows with some compared columns unequal: 33,924
Number of rows with all compared columns equal: 0

Column Comparison
-----------------

Number of columns compared with some values unequal: 36
Number of columns compared with all values equal: 10
Total number of values which compare unequal: 72,707

Columns with Unequal Values or Types
------------------------------------

                Column df1 dtype df2 dtype  # Unequal  